In [1]:
# Scrapping from reclameaqui.com.br
import pandas as pd
import time
import random
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import os
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import setuptools._distutils

# driver
driver = uc.Chrome()

empresas = ['euro17-credito', 'emprestimo-sim', 'up-p', 'losango', 'banco-sofisa', 'banco-bmg', 'coin-nacional', 'finanzero', 'konsi', 'cashme']

for empresa in empresas:

    # URL da pagina principal da empresa
    url_p = 'https://www.reclameaqui.com.br/empresa/' + empresa + '/'

    # URL da pagina de reclamações da empresa
    def url_r(page : int, type : str = 'EVALUATED') -> str:

        # https://www.reclameaqui.com.br/empresa/temu/lista-reclamacoes/?pagina=1&status=EVALUATED
        return 'https://www.reclameaqui.com.br/empresa/' + empresa + '/lista-reclamacoes/?pagina='+ str(page) +'&status=' + type


        # pegando metadados da empresa (nome, categoria, reputação, sobre)

    if not os.path.exists(f'metadados/metadados_{empresa}.csv'):
        # acessando a pagina principal da empresa
        driver.get(url_p)

        time.sleep(random.randint(4,6))
            
        # ramo de atividade da empresa
        categoria = driver.find_element(By.ID, 'info_segmento_hero').text

        # reputação da empresa
        reputacao = driver.find_element(By.ID, 'tag_reputacao_hero').text

        # sobre a empresa
        sobre = driver.find_element(By.CSS_SELECTOR, 'ul.sc-1915fv4-2.idgfcC').text
        sobre = sobre.split("\n")[0] 

        metadata = {
            'nome': empresa,
            'categoria': categoria,         
            'reputacao': reputacao,         
            'sobre': sobre       
        }
        
        metadata_p = pd.DataFrame(metadata, index=[0])
        metadata_p.to_csv(f'metadados/metadados_{empresa}.csv', index=False)

    else:
    
        metadata = pd.read_csv(f'metadados/metadados_{empresa}.csv')
        metadata = metadata.to_dict(orient='records')[0]
    
    
    print(metadata)


    # pegando hyperlinks das reclamações avaliadas e/ou respondidas

    driver.get(url_r(page=1, type='EVALUATED'))

    all_hyperlinks = []
    Numero_de_paginas = 50  # máximo de 50 páginas

    # Se o CSV ainda não existe, inicia a coleta
    if not os.path.exists(f'hyperlinks/hyperlinks_{empresa}.csv'):
    
        for i in range(Numero_de_paginas):
            try:
                # Captura os hyperlinks da página atual
                div_hyperlinks = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1pe7b5t-0.eJgBOc')
                current_hyperlinks = [h.find_element(By.CSS_SELECTOR, 'a').get_attribute('href') for h in div_hyperlinks]
                
                if not current_hyperlinks:
                    print("Nenhum hyperlink encontrado")
                    break

                print(f"Página {i+1} - hyperlinks: {current_hyperlinks}")
                all_hyperlinks += current_hyperlinks

                botao = driver.find_element(By.CSS_SELECTOR, "button[data-testid='next-page-navigation-button']")
                driver.execute_script("arguments[0].click();", botao)
                time.sleep(random.randint(5, 8))

                div_hyperlinks_next = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1pe7b5t-0.eJgBOc')
                next_hyperlinks = [h.find_element(By.CSS_SELECTOR, 'a').get_attribute('href') for h in div_hyperlinks_next]

                # Se os hyperlinks não mudaram, chegou à última página
                if next_hyperlinks == current_hyperlinks:
                    print("Última página alcançada")
                    break

            except Exception as e:
                print(f"Erro encontrado: {e}")
                break


        df = pd.DataFrame(all_hyperlinks, columns=["Hyperlinks"])
        df.to_csv(f'hyperlinks/hyperlinks_{empresa}.csv', index=False)
        all_hyperlinks = df["Hyperlinks"].tolist()
            
    else:

        # Se o CSV já existe, apenas o lê
        df = pd.read_csv(f'hyperlinks/hyperlinks_{empresa}.csv')
        all_hyperlinks = df["Hyperlinks"].tolist()      

    print(all_hyperlinks)



    # pegando dados das reclamações respondidas

    reclamacoes = []

    #for link in all_hyperlinks:
    for link in all_hyperlinks:
        link = link[0]
        
        try:
            driver.get(link)

            time.sleep(random.randint(5, 8))

            titulo = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-title"]').text
            lugar = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-location"]').text
            data = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-creation-date"]').text
            reclamacao_id = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-id"]').text
            
            reclamacao_id = (reclamacao_id.split('ID: ')[1])

            parsed_data = time.strptime(data, '%d/%m/%Y às %H:%M')

            reclamacao_base = driver.find_element(By.CSS_SELECTOR, '.sc-lzlu7c-17.fRVYjv').text
            interacoes = driver.find_elements(By.CSS_SELECTOR, '[data-testid="complaint-interaction"]')

            interacoes_text = [i.text for i in interacoes]

            again = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(2) > div').text
            rating = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(3) > div').text
            status = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-status"]').text

            rating = int(rating.split()[0])
            again = again == 'Sim'

            reclamacao = {
                'id': reclamacao_id,
                'titulo': titulo,
                'lugar': lugar,
                'data': parsed_data,
                'reclamacao_base': reclamacao_base,
                'interacoes': interacoes_text,
                'rating': rating,
                'again': again,
                'status': status
            }

            reclamacoes.append(reclamacao)
            

        except:
            
            continue

    print(reclamacoes)

    # transformando em csv as reclamações
    df = pd.DataFrame(reclamacoes)
    df.to_csv(f'reclamacoes/reclamacoes_{empresa}.csv', index=False)

   
driver.quit()



{'nome': 'euro17-credito', 'categoria': 'Bancos e Financeiras', 'reputacao': 'RA1000', 'sobre': 'A Euro17 Crédito oferece soluções de crédito pessoal, com e sem garantia, consignado, além de financiamento de bens móveis e imóveis.'}
Página 1 - hyperlinks: ['https://www.reclameaqui.com.br/euro17-credito/confirmar-autenticidade-do-funcionario_tPYWJkUCGoLWFwEz/', 'https://www.reclameaqui.com.br/euro17-credito/empresa_PynCUOUo347g1vTJ/', 'https://www.reclameaqui.com.br/euro17-credito/entraram-em-contato_5JDSx7iZSlE6NPSn/', 'https://www.reclameaqui.com.br/euro17-credito/validar-contatos_jik7YaEPoKmqTRXk/', 'https://www.reclameaqui.com.br/euro17-credito/esclarecimento_7iAjmmJ0SMbOfIAm/', 'https://www.reclameaqui.com.br/euro17-credito/confirmacao-de-contato_TyjtcxMz6l2cTYwt/', 'https://www.reclameaqui.com.br/euro17-credito/quero-receber-uma-confirmacao_FVC1w3CndBslEYNq/', 'https://www.reclameaqui.com.br/euro17-credito/sem-resposta_cNpBbM0HGpDWcx41/', 'https://www.reclameaqui.com.br/euro17-cre

OSError: Cannot save file into a non-existent directory: 'reclamacoes'